# 🔧 ALCHEMIST CONTROL — Debug & Testing

```
╔══════════════════════════════════════════════════════════════════════════════╗
║              FRÉGATE 05_ALCHEMIST — CONTROL PANEL                            ║
║             Debug • Test LUTs • Preview Effects • Inspect EXR                ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Objectif**: Tester individuellement chaque composant du pipeline ALCHEMIST.

---

## 📦 Setup & Mount Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/05_ALCHEMIST_LAB"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"

# Add codebase to path
import sys
sys.path.insert(0, CODEBASE)

print(f"✓ Drive monté")
print(f"✓ Unit Root: {UNIT_ROOT}")

In [ ]:
# Install dependencies
!pip install -q numpy scipy Pillow opencv-python-headless imageio tqdm omegaconf

## 🔍 Test 1: Vérification des Chemins

In [ ]:
from pathlib import Path

paths_to_check = [
    ("Unit Root", UNIT_ROOT),
    ("Codebase", CODEBASE),
    ("IN_RENDER", f"{UNIT_ROOT}/IN_RENDER"),
    ("OUT_GRADED", f"{UNIT_ROOT}/OUT_GRADED"),
    ("LUTS", f"{UNIT_ROOT}/LUTS"),
]

print("=== Vérification Chemins ===")
for name, path in paths_to_check:
    exists = Path(path).exists()
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path}")

In [ ]:
# Lister les fichiers disponibles
import os

print("\n=== Fichiers Inputs ===")

render_path = f"{UNIT_ROOT}/IN_RENDER"
print(f"\n📁 IN_RENDER:")
if Path(render_path).exists():
    files = [f for f in os.listdir(render_path) if not f.startswith('.')]
    exr_files = [f for f in files if f.endswith('.exr')]
    json_files = [f for f in files if f.endswith('.json')]
    print(f"   EXR files: {len(exr_files)}")
    print(f"   JSON files: {len(json_files)}")
    if exr_files[:5]:
        print(f"   Premiers EXR: {exr_files[:5]}")

luts_path = f"{UNIT_ROOT}/LUTS"
print(f"\n📁 LUTS:")
if Path(luts_path).exists():
    luts = [f for f in os.listdir(luts_path) if f.endswith('.cube')]
    for lut in luts:
        print(f"   - {lut}")

## 🎨 Test 2: Color Grader & LUTs

In [ ]:
from color_grader import ColorGrader, CubeLUTParser, CINEMATIC_PRESETS

print("=== Presets Cinématiques ===")
for name, preset in CINEMATIC_PRESETS.items():
    print(f"\n• {name}:")
    print(f"    Exposure: {preset.exposure:+.2f}")
    print(f"    Contrast: {preset.contrast:.2f}")
    print(f"    Saturation: {preset.saturation:.2f}")
    print(f"    Temperature: {preset.temperature:+.2f}")

In [ ]:
# Test LUT Parser
grader = ColorGrader(f"{UNIT_ROOT}/LUTS")

print("\n=== LUTs Disponibles ===")
available = grader.list_available()
print(f"Presets: {available['presets']}")
print(f"LUTs: {available['luts']}")

In [ ]:
# Test chargement LUT
lut = grader.load_lut("cinematic_warm")

if lut:
    print(f"\n=== LUT: {lut.title} ===")
    print(f"  Size: {lut.size}³")
    print(f"  Data points: {len(lut.data)}")
    print(f"  Lift (shadows): {[f'{v:.3f}' for v in lut.lift]}")
    print(f"  Gamma (midtones): {[f'{v:.3f}' for v in lut.gamma]}")
    print(f"  Gain (highlights): {[f'{v:.3f}' for v in lut.gain]}")
    print(f"  Temperature shift: {lut.temperature_shift:+.3f}")

## 🌟 Test 3: Effects Forge

In [ ]:
from effects_forge import EffectsForge, EFFECTS_PRESETS

forge = EffectsForge()

print("=== Effects Presets ===")
for name in forge.list_presets():
    preset = forge.get_preset(name)
    active = forge.get_active_effects(preset)
    print(f"\n• {name}:")
    print(f"    Effects: {', '.join(active) if active else 'none'}")

In [ ]:
# Test parsing PRODUCTION_PLAN effects
test_effects = {
    "bloom": True,
    "lens_flare": False,
    "film_grain": 0.08,
    "vignette": 0.25,
    "chromatic_aberration": 0.005
}

config = forge.parse_effects_dict(test_effects)

print("\n=== Parsed Effects ===")
print(f"Input: {test_effects}")
print(f"\nParsed:")
print(f"  Bloom: enabled={config.bloom.enabled}, intensity={config.bloom.intensity}")
print(f"  Lens Flare: enabled={config.lens_flare.enabled}")
print(f"  Film Grain: enabled={config.film_grain.enabled}, intensity={config.film_grain.intensity}")
print(f"  Vignette: enabled={config.vignette.enabled}, intensity={config.vignette.intensity}")
print(f"  Chromatic Aberration: enabled={config.chromatic_aberration.enabled}, dispersion={config.chromatic_aberration.dispersion}")

In [ ]:
# Générer les nodes Blender
nodes = forge.config_to_blender_nodes(config)

print("\n=== Blender Nodes à créer ===")
for node in nodes:
    print(f"  • {node['name']} ({node['type']})")
    for key, val in node.get('params', {}).items():
        print(f"      {key}: {val}")

## 🔇 Test 4: Denoiser

In [ ]:
from denoiser import Denoiser, DenoiserDetector, get_optimal_denoise_config

print("=== Détection Backends ===")

cuda = DenoiserDetector.detect_cuda()
print(f"  CUDA: {'✓' if cuda else '✗'}")

optix = DenoiserDetector.detect_optix()
print(f"  OptiX: {'✓' if optix else '✗'}")

oidn = DenoiserDetector.detect_oidn()
print(f"  OIDN: {'✓' if oidn else '✗'}")

blender = DenoiserDetector.detect_blender()
print(f"  Blender: {'✓' if blender else '✗'}")

best = DenoiserDetector.get_best_backend()
print(f"\n→ Backend recommandé: {best.value}")

In [ ]:
# Configuration optimale
config = get_optimal_denoise_config(
    has_albedo=False,
    has_normal=False,
    is_animation=True
)

print("\n=== Configuration Optimale ===")
print(f"  Backend: {config.backend.value}")
print(f"  HDR: {config.use_hdr}")
print(f"  Prefilter: {config.prefilter}")
print(f"  Preserve Detail: {config.preserve_detail}")

## 📊 Test 5: Inspection EXR

In [ ]:
from denoiser import validate_exr_for_denoise
from pathlib import Path
import os

render_dir = Path(f"{UNIT_ROOT}/IN_RENDER")
exr_files = list(render_dir.glob("*.exr"))

if exr_files:
    test_exr = str(exr_files[0])
    print(f"=== Inspection: {Path(test_exr).name} ===")
    
    result = validate_exr_for_denoise(test_exr)
    print(f"  Valid: {result['valid']}")
    print(f"  Has Color: {result['has_color']}")
    print(f"  Has Albedo: {result['has_albedo']}")
    print(f"  Has Normal: {result['has_normal']}")
    print(f"  Has Depth: {result['has_depth']}")
    if result['channels']:
        print(f"  Channels: {result['channels'][:10]}...")
    if result['error']:
        print(f"  Error: {result['error']}")
else:
    print("Aucun fichier EXR trouvé dans IN_RENDER/")

## 🚀 Test 6: Dry Run Pipeline

In [ ]:
# Test dry-run du script principal
!python {CODEBASE}/EXO_05_ALCHEMIST.py --help

In [ ]:
# Exemple de dry-run (validation chemins sans exécution)
# Décommenter et adapter le chemin du PRODUCTION_PLAN.JSON

# !python {CODEBASE}/EXO_05_ALCHEMIST.py \
#     --drive-root {DRIVE_ROOT} \
#     --production-plan PRODUCTION_PLAN.JSON \
#     --dry-run -v

## 🎬 Test 7: Preview Visuelle (Optional)

In [ ]:
# Visualisation d'un grade sur une image test
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from color_grader import apply_color_grade_values, ColorGradeConfig

# Créer une image gradient test
size = 256
gradient = np.zeros((size, size, 3), dtype=np.float32)

for y in range(size):
    for x in range(size):
        gradient[y, x] = [x/size, y/size, 0.5]

# Appliquer différents grades
presets_to_test = ['natural', 'cinematic_warm', 'cinematic_cold', 'neon_nights']

fig, axes = plt.subplots(1, len(presets_to_test), figsize=(16, 4))

for idx, preset_name in enumerate(presets_to_test):
    config = grader.get_config_from_name(preset_name)
    
    graded = np.zeros_like(gradient)
    for y in range(size):
        for x in range(size):
            r, g, b = gradient[y, x]
            graded[y, x] = apply_color_grade_values(r, g, b, config)
    
    axes[idx].imshow(graded)
    axes[idx].set_title(preset_name)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

---
## ✅ Checklist Debug

- [ ] Chemins vérifiés
- [ ] LUTs chargées correctement
- [ ] Effects parsing fonctionne
- [ ] Denoiser backend détecté
- [ ] Dry-run passe